<a href="https://colab.research.google.com/github/chierighini/notes/blob/main/python_notes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Part: https://colab.research.google.com/drive/1W2aWQHJgf2wOU7_xMqlXyoFMjvP9wIlO?usp=sharing

I made them separate notes for the sake of readability and so that GitHub would load the previews faster... not that I know if that actually works. I guess we all have our quirks.

#Meta Classes

Meta classes define the rules for creating a class. It is possible to make a meta class to change the way a class is created.

##Type of class
When checking for the object type of class we observe that it is of object ```type``` as opposed to the type of a function, an integer or even the instance of the same class:

In [ ]:
class C:
  def func(self):
    print('hello')

def func():
  pass

print(type(func))
print(type(5))
print(type(C()))
print(type(C))

<class 'function'>
<class 'int'>
<class '__main__.C'>
<class 'type'>


From this we learn that a class can be created using the ```type``` method, which takes the arguments as: ```type(class_name,(bases,),{'attribute':value})```. In the next example, the ```Test``` class inherits from the first class ```C``` and has two integer attributes ```x``` and ```y``` and also a method, which is defined beforehand. Those arguments are passed on to a ```type```class, which then turns it into an actual class.

In [ ]:
def aFunc(self):
  self.k = 15

Test = type('Test',(C,),{'x':5,'y':10,'aFunc':aFunc})

exmp=Test()

print(type(Test()))
print(exmp.x)
print(exmp.y)
exmp.func()
exmp.aFunc()
print(exmp.k)

<class '__main__.Test'>
5
10
hello
15


##The Meta classes themselves

Meta classes stand above the classes we create and can be used to modify the way that a regular class is created.

In [ ]:
class Meta(type):
  def __new__(self,class_name,bases,attrs):
    print(attrs)

    a = {}
    for name,value in attrs.items():
      if name.startswith("__"):
        a[name] = value
      else:
        a[name.upper()] = value
    print(a)
    return type(class_name,bases,a)

class A(metaclass=Meta):
  x=6
  k=7

print(A.X)

{'__module__': '__main__', '__qualname__': 'A', 'x': 6, 'k': 7}
{'__module__': '__main__', '__qualname__': 'A', 'X': 6, 'K': 7}
6


The example shown above uses a meta class to capitalize all attribute names that do not start with a double underscore, which you can notice by the difference between the ```print()``` methods inside the meta class and the ```print(A.X)``` statement.

##Quick note

I don't know if meta classes are good programming practice, and this section of my notes is here for the sake of knowing what a meta class is and how it works. 

#Decorators

Modifies the behavior of a function.Decorators can be used to add a functionality to the existing code without actually changing it.


In [18]:
def dec_func(f):
  def wrapper(*args,**kwargs):
    r = f(*args,**kwargs)
    print(r)
    return r
  return wrapper

@dec_func
def dec_func2(x):
  print("foo" + x)
  return "end"

In this example the ```dec_func(f)``` takes another function as an argument and passes it on to a wrapper, which will be executed when ```dec_func2(x)``` is called, together with its alredy existing instructions, As seen in the following lines:

In [20]:
k = dec_func2("bar")

foobar
end


In [21]:
print(k)

end


In [22]:
k

'end'

##Real life usage

A good example is when you need a timer functionality to see how long it took for your function to run, or a logging functionality to see what's being called and so on.

In [24]:
import time

In [26]:
def timer(f):
  def wrapper(*args,**kwargs):
    start = time.time()
    r = f(*args,**kwargs)
    total = time.time() - start
    print(total)
    return r
  return wrapper

@timer
def timed_function():
  for _ in range(99999999):
    pass

@timer
def timed_function1():
  time.sleep(2)

In [27]:
timed_function()
timed_function1()

2.4492151737213135
2.0022289752960205


#Generators

Generatos are a way to make values and not store them in memory, making your code more memory optimal. The first example is to show how the generator works by "making" it from scratch.

In [29]:
class Gen:
  def __init__(self, n):
    self.n = n
    self.last = 0

  def __next__(self):
    return self.next()

  def next(self):
    if self.last == self.n:
      raise StopIteration()
      
    rv = self.last ** 2
    self.last += 1
    return rv

In [31]:
g = Gen(15)

while True:
  try:
    print(next(g))
  except StopIteration:
    break

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196


The demonstration above is only there to understand the process of "generation". The ```Gen``` class defines an entry value and a last used value and uses them the next time the it is called.

For a true generator we must use the ```yield``` keyword to "pause" a function in its current value.

In [32]:
def gen(n):
  for i in range(n):
    yield i**2

g = gen(10)

for i in g:
  print(i)

0
1
4
9
16
25
36
49
64
81


We iterate through ```g``` in order to get the next value from the generator, however, we do not need to use a loop for that:

In [35]:
g = gen(4)

print(next(g))
print(next(g))
print(next(g))
print(next(g))
try:
  print(next(g))
except StopIteration:
  print("yep, that's an error")

0
1
4
9
yep, that's an error


And as you can see, printing more times than the "to be generated" amount gets you a ```StopIteration``` error, which we can then handle.

#Context Manager
Context Managers are a way to ensure an operation occurs in the event of a crash or an exit from somewhere in the code.

The simplest example would be to open a file and ensuring that it closes properly, which can be done with the use of the ```with``` context manager:
```
with open("file", "r") as file:
  file.write("hello")
```
In this case the ```open()``` method takes a filepath as a first argument and a mode in which the file will be opened.

The next example will be a context manager made from scratch to see the inner workings of it. Please disregard the weird vocabulary, it's just me trying to be funnny.

In [43]:
class File:
  def __init__(self,filename,method):
    self.file = open(filename,method)

  def __enter__(self):
    print("enter")
    return self.file

  def __exit__(self,type,value,traceback): #this dunder method is used for handling exceptions
    print(f"{type},{value},{traceback}")
    print("exit")
    self.file.close()
    if type == Exception:
      return True

In [49]:
with File("/content/file.txt","w") as f:
  print("middle")
  f.write("hello")
  raise Exception()

enter
middle
<class 'Exception'>,,<traceback object at 0x7fb9d7bdaf48>
exit


As you can see the exception that was raised was properly handled inside the ```__exit__``` dunder method and therefore did not cause any errors. If the return was anything different from true, it would have caused errors as well.

Now, using generators and decorators:

In [50]:
import contextlib

In [52]:
@contextlib.contextmanager
def file(filename,method):
  print("enter")
  file = open(filename,method)
  yield file
  file.close()
  print("exit")

In [53]:
with file("/content/file.txt","w") as f:
  print("middle")
  f.write("hello2")

enter
middle
exit


Both are valid ways of creating context managers and are very useful for openening files, or handling locks in case you're working with threading.

#Final Considerations

Some of theese topics were very difficult to understand at first, but thanks to this youtube playlist: https://www.youtube.com/watch?v=NAQEj-c2CI8&list=PLzMcBGfZo4-kwmIcMDdXSuy_wSqtU-xDP&index=3, it became extremely easier.

Kudos to the Tech With Tim Channel: https://www.youtube.com/channel/UC4JX40jDee_tINbkjycV4Sg, for making a very good advanced level python tutorial.

I believe I am not done with my python notes and probably will make more of them. I'll be sure to name my sources as much as I can so that this is not the only source of unlimited knowledge in the universe. All jokes aside, the sheer amount of snooping around for Python topics I have been doing have been paying off and I encourage anyone reading to seek the sources, so that misinformation can be avoided in the case of any mistakes I may have made.

#Sources
* https://www.youtube.com/watch?v=NAQEj-c2CI8&list=PLzMcBGfZo4-kwmIcMDdXSuy_wSqtU-xDP&index=3